# Dependences

In [1]:
include("../../src/struct_data.jl")
include("../../src/neighbor.jl")
include("../../src/forces/forces.jl")

cu_force

# Model & Aggregate

In [2]:
@time Model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 10000.0,
        dt    = 0.5,
        nₖₙₙ  = 50,
        nₛₐᵥₑ = 200
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "Test_1",
        path_output = ""
    )
)
dump(Model)

  0.000002 seconds (4 allocations: 144 bytes)
ModelSet
  Time: TimeModel
    tₛᵢₘ: Float64 10000.0
    dt: Float64 0.5
    nₖₙₙ: Int64 50
    nₛₐᵥₑ: Int64 200
  Input: InputModel
    outer_ratio: Float64 0.8
    path_input: String "../../data/init/Sphere"
  Output: OutputModel
    name_output: String "Test_1"
    path_output: String ""


In [3]:
Par1, Par2 = (0.005,2.0,3.0), 0.01

FusionAGG = nothing
# @time @start_agg FusionAGG = FusionAggregate(
@time FusionAGG = FusionAggregate(
    [
        AggType(
            "HEK_1", 
            InteractionPar(
                Cubic(Par1...), 
                ContractilePar(Par2)
            ),
            Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
        ),
        AggType(
            "HEK_2", 
            InteractionPar(
                Cubic(Par1...), 
                ContractilePar(Par2)
            ),
            Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
        )
    ], 
    Model
)

  7.196284 seconds (13.12 M allocations: 1.133 GiB, 3.61% gc time, 5.07% compilation time)


Aggregate(AggType[AggType("HEK_1", InteractionPar(Cubic{Float64}(0.005, 2.0, 3.0), ContractilePar(0.01)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}), AggType("HEK_2", InteractionPar(Cubic{Float64}(0.005, 2.0, 3.0), ContractilePar(0.01)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})], AggIndex([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2], ["HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"  …  "HEK_2", "HEK_2", "HEK_2", "HEK_2", "HEK_2", "HEK_2", "HEK_2", "HEK_2", "HEK_2", "HEK_2"]), Float32[-16.77 -4.62 -13.88; -14.77 -4.62 -13.88; … ; 15.77 4.62 13.88; 17.77 4.62 13.88], AggGeometry(Float32[15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27  …  15.27, 15.27, 15.27, 15.

# kNN & Force

In [4]:
println("--------------------------------- Running CUDA kNN ---------------------------------")
@time nearest_neighbors(FusionAGG)
display(FusionAGG.Simulation.Neighbor.idx_red)

--------------------------------- Running CUDA kNN ---------------------------------
 20.379677 seconds (31.23 M allocations: 1.775 GiB, 3.37% gc time, 18.42% compilation time)


21×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
  1   1   2   4   1   1   2   3   4  …  4949  4950  4951  4957  4958  4959
  2   2   3   5   4   2   3   7   9     4957  4958  4959  4964  4965  4966
  5   3   7   9   5   5   6   8  10     4958  4959  4960  4965  4966  4967
  6   6   8  10   6   6   7  13  15     4964  4965  4966  4971  4972  4973
 35   7  37  40  10   7   8  14  48     4965  4966  4967  4972  4973  4974
 36  36  38  41  11  11  12  44  49  …  4966  4967  4968  4973  4974  4975
 37  37  39  42  41  12  13  45  50     4997  4998  4999  5002  5003  5004
 42  38  44  49  42  42  43  46  57     4998  4999  5000  5003  5004  5005
 43  43  45  50  43  43  44  53  58     5002  5003  5004  5006  5006  5007
 51  44  53  58  50  44  45  54  66     5003  5004  5005  5007  5007  5008
  0  52   0   0  51  51  52  62   0  …  5004  5005  5008     0  5008     0
  0   0   0   0  59  52  53   0   0     5006  5007     0     0     0     0
  0   0   0   0   0  60  61   0   0     5007  5008

In [5]:
println("-------------------------------- Running CUDA Force --------------------------------")
@time cu_force(FusionAGG,Model,1.0)
display(FusionAGG.Position)

-------------------------------- Running CUDA Force --------------------------------
  1.665697 seconds (3.03 M allocations: 173.849 MiB, 2.37% gc time, 32.02% compilation time)


5008×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -16.77  -4.62  -13.88
 -14.77  -4.62  -13.88
 -12.77  -4.62  -13.88
 -19.77  -2.89  -13.88
 -17.77  -2.89  -13.88
 -15.77  -2.89  -13.88
 -13.77  -2.89  -13.88
 -11.77  -2.89  -13.88
 -20.77  -1.15  -13.88
 -18.77  -1.15  -13.88
 -16.77  -1.15  -13.88
 -14.77  -1.15  -13.88
 -12.77  -1.15  -13.88
   ⋮            
  13.77   1.15   13.88
  15.77   1.15   13.88
  17.77   1.15   13.88
  19.77   1.15   13.88
  10.77   2.89   13.88
  12.77   2.89   13.88
  14.77   2.89   13.88
  16.77   2.89   13.88
  18.77   2.89   13.88
  13.77   4.62   13.88
  15.77   4.62   13.88
  17.77   4.62   13.88

In [10]:
using ProgressMeter


# function simulate(agg::Aggregate, model::ModelSet, saving::Bool)
model = Model
agg = FusionAGG
# Calculating position of every cell on the fusion
@showprogress "Simulating Aggregate..." for step::Int = 0:1:(model.Time.tₛᵢₘ/model.Time.dt)
    
    # # Saving data in a given time (nₛₐᵥₑ)
    # if (step % model.Time.nₛₐᵥₑ+1) == 1
    #     save_append_data(
    #         agg, 
    #         step*model.Time.dt, 
    #         model.Output.path_output, 
    #         model.Output.name_output
    #     )
    # end

    # Calculating the kNN for the aggregate
    if (step % model.Time.nₖₙₙ+1) == 1
        nearest_neighbors(agg)
    end

    # # Compute the forces between each pair of particles in `agg` and their displacement.
    t_nₖₙₙ = step % model.Time.nₖₙₙ+1
    cu_force(agg,model,t_nₖₙₙ)
end